In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
project_name = "diff_flight_paths2"

In [3]:
import os
import numpy as np
import shapely
import src.base.connect_to_database as ctd
import src.load.load_image as li

conn = ctd.establish_connection()

# define project paths
PATH_PROJECT_FOLDERS = "/data/ATM/data_1/sfm/agi_projects"
project_fld = os.path.join(PATH_PROJECT_FOLDERS, project_name)
data_fld = os.path.join(project_fld, "data")
output_fld = os.path.join(project_fld, "output")

# get all image ids
image_fld = os.path.join(data_fld, "images")
images = [image.split(".")[0] for image in os.listdir(image_fld)]

# get all footprints
sql_string = "SELECT st_astext(footprint_exact) AS footprint_exact, azimuth_exact FROM images_georef WHERE image_id in ('{}')".format(
    "', '".join(images)
)
data = ctd.execute_sql(sql_string, conn)
# extract footprints 
footprints = [shapely.from_wkt(footprint) for footprint in data["footprint_exact"].tolist()]
# extract average azimuth
azimuth = np.mean(data['azimuth_exact'])

# create list with only TRUE values based on footprints
aligned = [False] * len(footprints)
for i in range(8):
    aligned[i] = True

# load ortho
ortho_path = os.path.join(output_fld, project_name + "_ortho_relative.tif")
ortho = li.load_image(ortho_path)

# get first band of ortho if it has more than one band
if len(ortho.shape) == 3:
    ortho = ortho[0]


In [8]:
import src.sfm_agi.snippets.georef_ortho as go

transform = go.georef_ortho(ortho, footprints, aligned, azimuth=azimuth)

(1042, 2)
(2, 3)
[-1698100.           892270.             6449.79638672     2545.83911133]


In [9]:
import src.georef.snippets.apply_transform as at

output_path = "/home/fdahle/Desktop/test_ortho.tif"

print(transform)

at.apply_transform(ortho, transform=transform, save_path=output_path)

[[      -2.26029098       -4.50566822 -1672097.37602715]
 [      -4.50543852        2.25634231   915671.87958412]
 [       0.                0.                1.        ]]
